In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

input_data = pd.read_excel(r'D:\Github repo\DataBase\signal_process\50-ETF-signalout.xlsx')
signal_input = pd.read_excel(r'D:\Github repo\DataBase\signal_process\signal_result-30 copy.xlsx') 
input_data.dropna(axis = 0, inplace= True)

# 加权处理，取四分位数进行分权
signal_input_quantile = signal_input[['annual_return_sp','win_rate_sp','pc_ratio_sp']].quantile([0.25,0.5,0.75])
for col_name in ['annual_return_sp','win_rate_sp','pc_ratio_sp']:
    def get_weight_group(x) :
    # 加权处理函数
        if x < signal_input_quantile[col_name].loc[0.25]: 
            return 1
        elif signal_input_quantile[col_name].loc[0.25] < x  and  x < signal_input_quantile[col_name].loc[0.5]: 
            return 2 
        elif signal_input_quantile[col_name].loc[0.5] < x and  x < signal_input_quantile[col_name].loc[0.75] : 
            return 3         
        else : 
            return 4 
    new_weighte_col_name = col_name + '_weight'
    signal_input[new_weighte_col_name] = signal_input[col_name].map(get_weight_group)

signal_input['Max_method'] = signal_input[['annual_return_sp','win_rate_sp','pc_ratio_sp']].sum(axis = 1)

signal_input['Max_method'].sort_values(ascending = False)

input_data



,trade_date,50ETF,2日开盘价持续下跌-1120-lr,2日开盘价持续下跌-1120-xgb,2日开盘价持续下跌-1120-rfc,2日开盘价持续下跌-1120-dtc,2日开盘价持续下跌-1120-knnc,2日开盘价持续下跌-1120-bagc,2日开盘价持续下跌-1121-lr,2日开盘价持续下跌-1121-xgb,...,top1-sharpe_ratio_op-60,top1-annual_return_op-60,top1-win_rate_op-60,top1-annual_return_sp-60,top1-sharpe_ratio_sp-60,top1-win_rate_sp-60,top1-pc_ratio_op-60,top1-pc_ratio_sp-60,top1-pc_ratio_op-30,top1-pc_ratio_sp-30
79,2015-05-05,3.149371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
80,2015-05-06,3.126536,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
81,2015-05-07,3.066530,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
82,2015-05-08,3.042063,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
83,2015-05-11,3.066016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,2020-12-25,3.493788,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1458,2020-12-28,3.519223,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,1,0,0,1,0,0,1,0
1459,2020-12-29,3.518002,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,1,0,0,1,0,0,1,0
1460,2020-12-30,3.538702,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,1,0,0,1,0,0,1,0


In [3]:

""" 当期相同则计算上期method
if len(signal_input_filter['Max_method']) > 3:       
    signal_input_filter.sort_values(by=['model','start_date'],inplace=True) 
    signal_input_filter['Last_method'] = signal_input_filter['Max_method'].shift(1) 
    signal_output = signal_input_filter.nlargest(3,'Last_method',keep="first")
else:
    signal_output = signal_input_filter.sort_values(by="start_date").shift(1)

    signal_input_filter.sort_values(by='start_date',inplace=True)
    signal_input_filter['model_selected'] = signal_input_filter['model'].shift(1)
    
    signal_input_filter.to_excel('/Users/dylan/DataBase/signal_process/1.xlsx',index=False) """


' 当期相同则计算上期method\nif len(signal_input_filter[\'Max_method\']) > 3:       \n    signal_input_filter.sort_values(by=[\'model\',\'start_date\'],inplace=True) \n    signal_input_filter[\'Last_method\'] = signal_input_filter[\'Max_method\'].shift(1) \n    signal_output = signal_input_filter.nlargest(3,\'Last_method\',keep="first")\nelse:\n    signal_output = signal_input_filter.sort_values(by="start_date").shift(1)\n\n    signal_input_filter.sort_values(by=\'start_date\',inplace=True)\n    signal_input_filter[\'model_selected\'] = signal_input_filter[\'model\'].shift(1)\n    \n    signal_input_filter.to_excel(\'/Users/dylan/DataBase/signal_process/1.xlsx\',index=False) '

In [13]:
# 选出当期Top3

signal_input_filter = signal_input.groupby(by = 'start_date').apply(lambda x : x.nlargest(3, 'Max_method', keep = 'first'))

# 在Top3中随机选择
""" signal_output = pd.DataFrame()
for i in signal_input_filter['start_date'].drop_duplicates(): 
    column = signal_input_filter.loc[(i)].sample(1)
    signal_output = signal_output.append(column)

signal_output['model_selected'] = signal_output['model'].shift(1) """

# 对Top3 进行处理 综合信号




" signal_output = pd.DataFrame()\nfor i in signal_input_filter['start_date'].drop_duplicates(): \n    column = signal_input_filter.loc[(i)].sample(1)\n    signal_output = signal_output.append(column)\n\nsignal_output['model_selected'] = signal_output['model'].shift(1) "

In [14]:
# 归一化
""" for m in signal_input_filter['start_date'].drop_duplicates():
    for n in signal_input_filter['start_date'].drop_duplicates():
    M1 = signal_input_filter.loc[i].rename(columns = {'annual_return_sp' : 'a','win_rate_sp' : 'b','pc_ratio_sp' : 'c'}, inplace = False)
    M2 = signal_input_filter.loc[i].rename(columns = {'annual_return_sp_weight' : 'a','win_rate_sp_weight' : 'b','pc_ratio_sp_weight' : 'c'}, inplace = False)
    signal_output = M1[['a','b','c']].dot(M2[['a','b','c']].T).T.iloc[0]
    Max = signal_output.max()
    Min = signal_output.min()
    signal_output = (signal_output - Min)/(Max - Min)
    signal_input['trade_date' > m and 'trade_date' < n ]['top_annual_return_sp-30','top_win_rate_sp-30','top_pc_ratio_sp-30']
"""
                                                                                        
M1 = signal_input_filter.loc['2015-07-01'].rename(columns = {'annual_return_sp' : 'a','win_rate_sp' : 'b','pc_ratio_sp' : 'c'}, inplace = False)

M2 = signal_input_filter.loc['2015-07-01'].rename(columns = {'annual_return_sp_weight' : 'a','win_rate_sp_weight' : 'b','pc_ratio_sp_weight' : 'c'}, inplace = False)

signal_output = M1[['a','b','c']].dot(M2[['a','b','c']].T).T.iloc[0]
Max = signal_output.max()
Min = signal_output.min()
signal_output = (signal_output - Min)/(Max - Min)


""" ['top_annual_return_sp-30','top_win_rate_sp-30','top_pc_ratio_sp-30'] """


,trade_date,50ETF,2日开盘价持续下跌-1120-lr,2日开盘价持续下跌-1120-xgb,2日开盘价持续下跌-1120-rfc,2日开盘价持续下跌-1120-dtc,2日开盘价持续下跌-1120-knnc,2日开盘价持续下跌-1120-bagc,2日开盘价持续下跌-1121-lr,2日开盘价持续下跌-1121-xgb,...,top1-sharpe_ratio_op-60,top1-annual_return_op-60,top1-win_rate_op-60,top1-annual_return_sp-60,top1-sharpe_ratio_sp-60,top1-win_rate_sp-60,top1-pc_ratio_op-60,top1-pc_ratio_sp-60,top1-pc_ratio_op-30,top1-pc_ratio_sp-30
79,2015-05-05,3.149371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
80,2015-05-06,3.126536,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
81,2015-05-07,3.066530,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
82,2015-05-08,3.042063,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
83,2015-05-11,3.066016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,2020-12-25,3.493788,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1458,2020-12-28,3.519223,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,1,0,0,1,0,0,1,0
1459,2020-12-29,3.518002,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,1,0,0,1,0,0,1,0
1460,2020-12-30,3.538702,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,1,0,0,1,0,0,1,0


In [16]:
""" pd.options.display.max_columns = None 
pd.options.display.max_rows = None
test1 = pd.to_datetime(input_data['trade_date']).le(pd.to_datetime(signal_input_filter['start_date']),axis = 0,level = 1) 

test2 = pd.to_datetime(input_data['trade_date']).ge(pd.to_datetime(signal_input_filter['start_date']),axis = 0,level = 1)
test1 & test2  """
    
for start_date in signal_input_filter['start_date'].drop_duplicates():
    for end_date in signal_input_filter['end_date'].drop_duplicates() :  
        input_data_filter = input_data[(start_date <= input_data['trade_date']) & (end_date >= input_data['trade_date'])]
        for row in input_data_filter.iloc[:] :
            row[['top1-annual_return_sp-30','top1-win_ratio_sp-30','pc_ratio_sp-30']]      

start_date      
2015-07-01  1004    2015-07-01
2015-08-12  1057    2015-08-12
2015-09-25  327     2015-09-25
2015-11-13  1281    2015-11-13
2015-12-25  1307    2015-12-25
2016-02-15  0       2016-02-15
2016-03-28  133     2016-03-28
2016-05-11  51      2016-05-11
2016-06-24  64      2016-06-24
2016-08-05  47      2016-08-05
2016-09-20  277     2016-09-20
2016-11-08  1459    2016-11-08
2016-12-20  50      2016-12-20
2017-02-08  63      2017-02-08
2017-03-22  1696    2017-03-22
2017-05-08  3       2017-05-08
2017-06-21  59      2017-06-21
2017-08-02  257     2017-08-02
2017-09-13  17      2017-09-13
2017-11-01  973     2017-11-01
2017-12-13  24      2017-12-13
2018-01-25  1009    2018-01-25
2018-03-15  397     2018-03-15
2018-05-02  22      2018-05-02
2018-06-13  27      2018-06-13
2018-07-26  218     2018-07-26
2018-09-06  278     2018-09-06
2018-10-26  1047    2018-10-26
2018-12-07  1371    2018-12-07
2019-01-22  450     2019-01-22
2019-03-12  66      2019-03-12
2019-04-24  1       20

In [10]:
### 得到信号

        
df1 = signal_output[['annual_return_sp','win_rate_sp','pc_ratio_sp']]
df3 = pd.DataFrame()
for x in signal_output['model']:
    df2 = signal_output[(x == signal_input['model'])]
    for y in signal_output['start_date']: 
        new_column = df2[y == df2['start_date']]
        df3 = df3.append(new_column)

df4 = df3.drop_duplicates('start_date')
df4.rename(columns = {'annual_return_sp_weight' : 'a','win_rate_sp_weight' : 'b','pc_ratio_sp_weight' : 'c'}, inplace = True)
df1.rename(columns = {'annual_return_sp' : 'a','win_rate_sp' : 'b','pc_ratio_sp' : 'c'}, inplace = True)
signal_output['signal'] = df1.dot(df4[['a','b','c']].T)

Max = signal_output['signal'].max()
Min = signal_output['signal'].min()
signal_output['signal'] = (signal_output['signal'] - Min)/(Max - Min)

/




NameError: name 'signal_output' is not defined

In [11]:
import pandas as pd
signal_input = pd.read_excel('D:\Github repo\DataBase\signal_process\signal_result-30 copy.xlsx')

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def cal_signal(methods,try_window,select_number) :
    # methods 以list形式传入
    
    input_data = pd.read_excel(r'D:\Github repo\DataBase\signal_process\50-ETF-signalout.xlsx')
    signal_input = pd.read_excel(r'\Github repo\DataBase\signal_process\signal_result-'+ str(try_window) +' copy.xlsx') 
    input_data.dropna(axis = 1, inplace= True)
    # 加权处理，取四分位数进行分权
    signal_input_quantile = signal_input[methods].quantile([0.25,0.5,0.75])
    for col_name in methods:
        def get_weight_group(x) :
            # 加权处理函数
            if x < signal_input_quantile[col_name].loc[0.25]: 
                return 1
            elif signal_input_quantile[col_name].loc[0.25] < x  and  x < signal_input_quantile[col_name].loc[0.5]: 
                return 2 
            elif signal_input_quantile[col_name].loc[0.5] < x and  x < signal_input_quantile[col_name].loc[0.75] : 
                return 3         
            else : 
                return 4 
        new_weighte_col_name = col_name + '_weight'
        signal_input[new_weighte_col_name] = signal_input[col_name].map(get_weight_group)
    signal_input['Max_method'] = signal_input[methods].sum(axis = 1)
    signal_input['Max_method'].sort_values(ascending = False)
    # 选出当期Top3
    signal_input_filter = signal_input.groupby(by = 'start_date').apply(lambda x : x.nlargest(3, 'Max_method', keep = 'first'))
    # 在Top3中进行随机选择
    signal_output = pd.DataFrame()
    for i in signal_input_filter['start_date'].drop_duplicates(): 
        column = signal_input_filter.loc[(i)].sample(select_number)
        signal_output = signal_output.append(column)
        signal_output['model_selected'] = signal_output['model'].shift(1)
        
    signal_output.to_excel(r'D:\Github repo\DataBase\signal_process\signal_output.xlsx')
        
    
cal_signal(methods,try_window,select_number)

NameError: name 'methods' is not defined

In [ ]:
'''
将各个评估指标进行分位数打分，选择top1-model
'''
def get_group_sprank(x):
    
    x['sp_rank_1'] = x['sharpe_ratio_sp'].rank(method='average',ascending=True)/len(x)
    x['sp_rank_2'] = x['win_rate_sp'].rank(method='average',ascending=True)/len(x)
    x['sp_rank_3'] = x['pc_ratio_sp'].rank(method='average',ascending=True)/len(x)
    
    x['sp_rank'] = ( x['sp_rank_1'] + x['sp_rank_2'] + x['sp_rank_3'] ) /3
    
    return (x['sp_rank'])

def cal_model_weight(x):
    
    tmp_dict = {}
    score_sum = sum(x['sp_rank'])
    for index,row in x.iterrows():        
        tmp_model = row['model']
        tmp_score = round(row['sp_rank']/score_sum,2)
        tmp_dict[tmp_model] = tmp_score

    return(tmp_dict)    
    

def cal_signal_byrank(training_window,select_number=1,thresh=0.5) :

    input_data = pd.read_excel('./input_data/50ETF-signalout.xlsx')
    signal_input = pd.read_excel('./input_data/signal_result-'+str(training_window)+'.xlsx')    
    #input_data.dropna(axis = 1, inplace= True)
    tmp_signal_name = 'top' + str(select_number) + '-' + 'sp_rank' + '-' + str(training_window)
    
    signal_input['sp_rank'] = signal_input.groupby("start_date").apply(get_group_sprank).reset_index(drop=True)
    
    # 选出当期TopN
    signal_input_filter = signal_input.groupby('start_date').apply(lambda x : x.nlargest(select_number,'sp_rank',keep = 'first')).reset_index(drop=True)
    #如果选择top1,将当期top_model传入下个回测期作为model_selected
    if select_number ==1:
        
        #将选出的top1 model错期一期作为下期信号
        signal_input_filter.sort_values(by='start_date',inplace=True)
        signal_input_filter['model_selected'] = signal_input_filter['model'].shift(1)
        
        signal_input_filter.to_excel('./input_data/signal_roll/'+tmp_signal_name+'.xlsx',index=False)
        
        input_data[tmp_signal_name] = 0
        for i in range(len(input_data)):
            tmp_trade_date = input_data['trade_date'].iloc[i]
            try:
                tmp_model = signal_input_filter[(signal_input_filter['start_date']<=tmp_trade_date)&(signal_in